In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch.optim as optim

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '58 tGravityAcc-energy()-Y',
 '59 tGravityAcc-energy()-Z',
 '128 tBodyGyro-mad()-Y',
 '141 tBodyGyro-iqr()-Y',
 '428 fBodyGyro-std()-Y',
 '434 fBodyGyro-max()-Y',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '483 fBodyGyro-bandsEnergy()-1,16',
 '487 fBodyGyro-bandsEnergy()-1,24',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '7 tBodyAcc-mad()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '202 tBodyAccMag-std()',
 '203 tBodyAccMag-mad()',
 '215 tGravityAccMag-std()',
 '216 tGravityAccMag-mad()',
 '266 fBodyAcc-mean()-X',
 '269 fBodyAcc-std()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '382 fBodyAccJerk-bandsEnergy()-1,8',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 30),
            classifier_block(30, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            nn.Linear(15, 15)
        )
    def forward(self, x):
        return self.network(x)

In [4]:
#defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = input_shape, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 80),
            generator_block(80, 60),
            generator_block(60, 50),
            nn.Linear(50, feature_dim),
            nn.Tanh()
        )
    def forward(self, noise):
        return self.gen(noise)

def load_model(model, model_name):
    model.load_state_dict(torch.load(f'../../../saved_models/{model_name}'))

In [5]:
#label is a list of integers specifying which labels to filter by
#users is a list of integers specifying which users to filter by
#y_label is a string, either "Activity" or "Subject" depending on what y output needs to be returned
def start_data(label, users, y_label, sub_features, act_features):
    #get the dataframe column names
    name_dataframe = pd.read_csv('../../../data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('../../../data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names
    
    X_train_1 = data[sub_features]
    X_train_2 = data[act_features]
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('../../../data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']
    
    y_train_subject = pd.read_csv('../../../data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']
    
    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X_1 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_2 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_3 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_4 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_5 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_6 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_7 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_8 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_9 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_10 = GAN_data[(GAN_data['Subject'] == 7) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_11 = GAN_data[(GAN_data['Subject'] == 7) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_12 = GAN_data[(GAN_data['Subject'] == 7) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_13 = GAN_data[(GAN_data['Subject'] == 8) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_14 = GAN_data[(GAN_data['Subject'] == 8) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_15 = GAN_data[(GAN_data['Subject'] == 8) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    
    X_train = np.concatenate((X_1, X_2, X_3, X_4, X_5, X_6, X_7, X_8, X_9, X_10, X_11, X_12, X_13, X_14, X_15))
    y_train = [0] * len(X_1) + [1] * len(X_2) + [2] * len(X_3) + [3] * len(X_4) + [4] * len(X_5) + [5] * len(X_6) + [6] * len(X_7) + [7] * len(X_8) + [8] * len(X_9) + [9] * len(X_10) + [10] * len(X_11) + [11] * len(X_12) + [12] * len(X_13) + [13] * len(X_14) + [14] * len(X_15)
    
    return X_train, np.asarray(y_train)

In [6]:
activities = [1, 3, 4]
users = [1, 3, 5, 7, 8]

X, y = start_data(activities, users, "Activity", sub_features, act_features)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model = Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [7]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 8.177769184112549, Final Batch Loss: 2.7224531173706055
Epoch 2, Loss: 8.167686462402344, Final Batch Loss: 2.7219529151916504
Epoch 3, Loss: 8.15138554573059, Final Batch Loss: 2.705904722213745
Epoch 4, Loss: 8.15267276763916, Final Batch Loss: 2.7183644771575928
Epoch 5, Loss: 8.139990329742432, Final Batch Loss: 2.7069091796875
Epoch 6, Loss: 8.133266687393188, Final Batch Loss: 2.717188835144043
Epoch 7, Loss: 8.126067638397217, Final Batch Loss: 2.7089264392852783
Epoch 8, Loss: 8.109956502914429, Final Batch Loss: 2.709545850753784
Epoch 9, Loss: 8.093474864959717, Final Batch Loss: 2.6938278675079346
Epoch 10, Loss: 8.07895040512085, Final Batch Loss: 2.680131196975708
Epoch 11, Loss: 8.040251016616821, Final Batch Loss: 2.658381700515747
Epoch 12, Loss: 8.016799926757812, Final Batch Loss: 2.659492015838623
Epoch 13, Loss: 8.01409125328064, Final Batch Loss: 2.6736557483673096
Epoch 14, Loss: 7.965330600738525, Final Batch Loss: 2.6631548404693604
Epoch 15, Loss

Epoch 128, Loss: 3.4488022327423096, Final Batch Loss: 1.1662291288375854
Epoch 129, Loss: 3.360398769378662, Final Batch Loss: 1.1469480991363525
Epoch 130, Loss: 3.459819197654724, Final Batch Loss: 1.1565370559692383
Epoch 131, Loss: 3.3738791942596436, Final Batch Loss: 1.1661906242370605
Epoch 132, Loss: 3.3577369451522827, Final Batch Loss: 1.122756004333496
Epoch 133, Loss: 3.361010432243347, Final Batch Loss: 1.1573371887207031
Epoch 134, Loss: 3.214348077774048, Final Batch Loss: 1.0492806434631348
Epoch 135, Loss: 3.366024613380432, Final Batch Loss: 1.089063286781311
Epoch 136, Loss: 3.416367292404175, Final Batch Loss: 1.2142877578735352
Epoch 137, Loss: 3.340568423271179, Final Batch Loss: 1.1397546529769897
Epoch 138, Loss: 3.4205946922302246, Final Batch Loss: 1.1145747900009155
Epoch 139, Loss: 3.3296287059783936, Final Batch Loss: 1.1089563369750977
Epoch 140, Loss: 3.320494294166565, Final Batch Loss: 1.1232314109802246
Epoch 141, Loss: 3.244964122772217, Final Batch 

Epoch 240, Loss: 2.737009108066559, Final Batch Loss: 0.9028817415237427
Epoch 241, Loss: 2.7322688698768616, Final Batch Loss: 0.8967946767807007
Epoch 242, Loss: 2.7549678087234497, Final Batch Loss: 0.9510049223899841
Epoch 243, Loss: 2.6720088720321655, Final Batch Loss: 0.8099027276039124
Epoch 244, Loss: 2.7036232352256775, Final Batch Loss: 0.9524038434028625
Epoch 245, Loss: 2.7093843817710876, Final Batch Loss: 0.9121677279472351
Epoch 246, Loss: 2.8274841904640198, Final Batch Loss: 0.9547707438468933
Epoch 247, Loss: 2.8006219267845154, Final Batch Loss: 0.95805823802948
Epoch 248, Loss: 2.6386460661888123, Final Batch Loss: 0.808814287185669
Epoch 249, Loss: 2.6489736437797546, Final Batch Loss: 0.8418446779251099
Epoch 250, Loss: 2.6481334567070007, Final Batch Loss: 0.8883662819862366
Epoch 251, Loss: 2.7247522473335266, Final Batch Loss: 0.8031099438667297
Epoch 252, Loss: 2.6531792283058167, Final Batch Loss: 0.8879942297935486
Epoch 253, Loss: 2.8381585478782654, Final

Epoch 360, Loss: 2.443763494491577, Final Batch Loss: 0.7702013850212097
Epoch 361, Loss: 2.5993043184280396, Final Batch Loss: 0.8261994123458862
Epoch 362, Loss: 2.397452652454376, Final Batch Loss: 0.6920777559280396
Epoch 363, Loss: 2.4247010350227356, Final Batch Loss: 0.7210518717765808
Epoch 364, Loss: 2.385859787464142, Final Batch Loss: 0.789771318435669
Epoch 365, Loss: 2.4677241444587708, Final Batch Loss: 0.8002732992172241
Epoch 366, Loss: 2.4676854014396667, Final Batch Loss: 0.7766843438148499
Epoch 367, Loss: 2.371144413948059, Final Batch Loss: 0.708256185054779
Epoch 368, Loss: 2.40841007232666, Final Batch Loss: 0.7706584930419922
Epoch 369, Loss: 2.503638803958893, Final Batch Loss: 0.7853737473487854
Epoch 370, Loss: 2.5132779479026794, Final Batch Loss: 0.8663626313209534
Epoch 371, Loss: 2.4252877831459045, Final Batch Loss: 0.8627345561981201
Epoch 372, Loss: 2.5336243510246277, Final Batch Loss: 0.8733600378036499
Epoch 373, Loss: 2.32599538564682, Final Batch 

Epoch 487, Loss: 2.3509780764579773, Final Batch Loss: 0.8077635765075684
Epoch 488, Loss: 2.2793740034103394, Final Batch Loss: 0.8066166639328003
Epoch 489, Loss: 2.28692227602005, Final Batch Loss: 0.6711451411247253
Epoch 490, Loss: 2.338951826095581, Final Batch Loss: 0.6983197927474976
Epoch 491, Loss: 2.3311960101127625, Final Batch Loss: 0.9151055812835693
Epoch 492, Loss: 2.204559624195099, Final Batch Loss: 0.7171642780303955
Epoch 493, Loss: 2.346126616001129, Final Batch Loss: 0.7988313436508179
Epoch 494, Loss: 2.3858678340911865, Final Batch Loss: 0.7599321007728577
Epoch 495, Loss: 2.2736161947250366, Final Batch Loss: 0.7483795881271362
Epoch 496, Loss: 2.324258506298065, Final Batch Loss: 0.8467038869857788
Epoch 497, Loss: 2.269140839576721, Final Batch Loss: 0.7061317563056946
Epoch 498, Loss: 2.2995938062667847, Final Batch Loss: 0.7984402775764465
Epoch 499, Loss: 2.3305631279945374, Final Batch Loss: 0.8118811845779419
Epoch 500, Loss: 2.2971271872520447, Final Ba

...

Epoch 4461, Loss: 0.7379040718078613, Final Batch Loss: 0.1760009378194809
Epoch 4462, Loss: 0.8325831145048141, Final Batch Loss: 0.3352440595626831
Epoch 4463, Loss: 0.7436998039484024, Final Batch Loss: 0.2553139626979828
Epoch 4464, Loss: 0.8465525209903717, Final Batch Loss: 0.3268398940563202
Epoch 4465, Loss: 0.8161707073450089, Final Batch Loss: 0.2799432575702667
Epoch 4466, Loss: 0.7917259335517883, Final Batch Loss: 0.2616164982318878
Epoch 4467, Loss: 0.7378311008214951, Final Batch Loss: 0.2661806643009186
Epoch 4468, Loss: 0.7832795977592468, Final Batch Loss: 0.2275277078151703
Epoch 4469, Loss: 0.7257529944181442, Final Batch Loss: 0.24828213453292847
Epoch 4470, Loss: 0.8859237730503082, Final Batch Loss: 0.2909923493862152
Epoch 4471, Loss: 0.724128857254982, Final Batch Loss: 0.21791143715381622
Epoch 4472, Loss: 0.8489443957805634, Final Batch Loss: 0.2895049750804901
Epoch 4473, Loss: 0.7855512052774429, Final Batch Loss: 0.247922882437706
Epoch 4474, Loss: 0.74394

Epoch 4572, Loss: 0.7330328077077866, Final Batch Loss: 0.2365954965353012
Epoch 4573, Loss: 0.7809027135372162, Final Batch Loss: 0.2967871129512787
Epoch 4574, Loss: 0.776068925857544, Final Batch Loss: 0.19644354283809662
Epoch 4575, Loss: 0.7861611396074295, Final Batch Loss: 0.2513754069805145
Epoch 4576, Loss: 0.8574210107326508, Final Batch Loss: 0.3522956371307373
Epoch 4577, Loss: 0.7273855954408646, Final Batch Loss: 0.2369120568037033
Epoch 4578, Loss: 0.7382264882326126, Final Batch Loss: 0.23013606667518616
Epoch 4579, Loss: 0.8238526582717896, Final Batch Loss: 0.30826815962791443
Epoch 4580, Loss: 0.8190379589796066, Final Batch Loss: 0.3067404329776764
Epoch 4581, Loss: 0.6988376080989838, Final Batch Loss: 0.176228329539299
Epoch 4582, Loss: 0.7117886543273926, Final Batch Loss: 0.2638768255710602
Epoch 4583, Loss: 0.7188486307859421, Final Batch Loss: 0.2486567199230194
Epoch 4584, Loss: 0.7578462362289429, Final Batch Loss: 0.2626374661922455
Epoch 4585, Loss: 0.6961

Epoch 4691, Loss: 0.8444871306419373, Final Batch Loss: 0.21690627932548523
Epoch 4692, Loss: 0.7656484395265579, Final Batch Loss: 0.29711419343948364
Epoch 4693, Loss: 0.8055001050233841, Final Batch Loss: 0.2700493633747101
Epoch 4694, Loss: 0.735363095998764, Final Batch Loss: 0.22505082190036774
Epoch 4695, Loss: 0.8185482025146484, Final Batch Loss: 0.263126403093338
Epoch 4696, Loss: 0.6819024682044983, Final Batch Loss: 0.2488558143377304
Epoch 4697, Loss: 0.6984629184007645, Final Batch Loss: 0.24421273171901703
Epoch 4698, Loss: 0.8375874310731888, Final Batch Loss: 0.24571995437145233
Epoch 4699, Loss: 0.7607681751251221, Final Batch Loss: 0.2773018479347229
Epoch 4700, Loss: 0.7672654241323471, Final Batch Loss: 0.26309528946876526
Epoch 4701, Loss: 0.7139180302619934, Final Batch Loss: 0.23584917187690735
Epoch 4702, Loss: 0.6609308123588562, Final Batch Loss: 0.23575164377689362
Epoch 4703, Loss: 0.7687216103076935, Final Batch Loss: 0.2158411145210266
Epoch 4704, Loss: 0

Epoch 4812, Loss: 0.7085794806480408, Final Batch Loss: 0.21913111209869385
Epoch 4813, Loss: 0.8365488350391388, Final Batch Loss: 0.35914885997772217
Epoch 4814, Loss: 0.7590373158454895, Final Batch Loss: 0.2718757688999176
Epoch 4815, Loss: 0.7911156862974167, Final Batch Loss: 0.2911188006401062
Epoch 4816, Loss: 0.8550262302160263, Final Batch Loss: 0.23256663978099823
Epoch 4817, Loss: 0.830237939953804, Final Batch Loss: 0.35051438212394714
Epoch 4818, Loss: 0.8184681236743927, Final Batch Loss: 0.24284303188323975
Epoch 4819, Loss: 0.7103334069252014, Final Batch Loss: 0.1658416986465454
Epoch 4820, Loss: 0.8195820599794388, Final Batch Loss: 0.3465299904346466
Epoch 4821, Loss: 0.7308111488819122, Final Batch Loss: 0.2802888751029968
Epoch 4822, Loss: 0.6844441741704941, Final Batch Loss: 0.23080819845199585
Epoch 4823, Loss: 0.7576818615198135, Final Batch Loss: 0.24756354093551636
Epoch 4824, Loss: 0.7255326807498932, Final Batch Loss: 0.24702109396457672
Epoch 4825, Loss: 

Epoch 4921, Loss: 0.7148543149232864, Final Batch Loss: 0.2487582117319107
Epoch 4922, Loss: 0.7267736047506332, Final Batch Loss: 0.22472982108592987
Epoch 4923, Loss: 0.7244439721107483, Final Batch Loss: 0.247938334941864
Epoch 4924, Loss: 0.7598877549171448, Final Batch Loss: 0.30810827016830444
Epoch 4925, Loss: 0.7350207269191742, Final Batch Loss: 0.26926642656326294
Epoch 4926, Loss: 0.7252707034349442, Final Batch Loss: 0.21072803437709808
Epoch 4927, Loss: 0.7860008031129837, Final Batch Loss: 0.24512846767902374
Epoch 4928, Loss: 0.7593415379524231, Final Batch Loss: 0.244456484913826
Epoch 4929, Loss: 0.9252462983131409, Final Batch Loss: 0.33915936946868896
Epoch 4930, Loss: 0.8831250220537186, Final Batch Loss: 0.39524295926094055
Epoch 4931, Loss: 0.6563548296689987, Final Batch Loss: 0.22554779052734375
Epoch 4932, Loss: 0.7673037201166153, Final Batch Loss: 0.27026599645614624
Epoch 4933, Loss: 0.6420511454343796, Final Batch Loss: 0.21909216046333313
Epoch 4934, Loss:

In [8]:
softmax = nn.Softmax(dim = 1)
model.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[18  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  9  0  0  0  0  0  0  0  0  0  0  0  1  0]
 [ 0  0  5  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 13  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 11  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  2  0  0  4  0  0  4  0  0  0  0  0  2]
 [ 0  0  0  0  0  0 11  1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  9  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  5  0  0  0  0  0  5]
 [ 0  0  0  0  0  1  0  0  0 12  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  7  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 12  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  7  0  0]
 [ 1  1  0  0  0  0  0  0  0  0  1  0  0  5  0]
 [ 0  0  2  0  0  3  0  0  0  0  0  0  0  0  5]]
              precision    recall  f1-score   support

           0    0.94737   1.00000   0.97297        18
           1    0.90000   0.90000   0.90000        10
           2    0.55556   1.00000   0.71429         5
           3    1.00000   1.00000   1.00000        13
        

In [9]:
n_samples = 20

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_1 = gen(latent_vectors).detach().numpy()

y_1 = np.zeros(n_samples)

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_2 = gen(latent_vectors).detach().numpy()

y_2 = np.ones(n_samples)

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U0A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_3 = gen(latent_vectors).detach().numpy()

y_3 = np.ones(n_samples) + 1

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_4 = gen(latent_vectors).detach().numpy()

y_4 = np.ones(n_samples) + 2

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_5 = gen(latent_vectors).detach().numpy()

y_5 = np.ones(n_samples) + 3

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U1A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_6 = gen(latent_vectors).detach().numpy()

y_6 = np.ones(n_samples) + 4

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_7 = gen(latent_vectors).detach().numpy()

y_7 = np.ones(n_samples) + 5

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_8 = gen(latent_vectors).detach().numpy()

y_8 = np.ones(n_samples) + 6

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U2A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_9 = gen(latent_vectors).detach().numpy()

y_9 = np.ones(n_samples) + 7

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U3A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_10 = gen(latent_vectors).detach().numpy()

y_10 = np.ones(n_samples) + 8

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U3A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_11 = gen(latent_vectors).detach().numpy()

y_11 = np.ones(n_samples) + 9

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U3A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_12 = gen(latent_vectors).detach().numpy()

y_12 = np.ones(n_samples) + 10

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U4A0 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_13 = gen(latent_vectors).detach().numpy()

y_13 = np.ones(n_samples) + 11

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U4A1 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_14 = gen(latent_vectors).detach().numpy()

y_14 = np.ones(n_samples) + 12

gen = Generator(z_dim = 100)
gen.eval()
load_model(gen, "U4A2 Solo GAN Ablation_gen.param")
latent_vectors = get_noise(n_samples, 100)
fake_features_15 = gen(latent_vectors).detach().numpy()

y_15 = np.ones(n_samples) + 13


fake_features = np.concatenate((fake_features_1, fake_features_2, fake_features_3, fake_features_4, fake_features_5, fake_features_6,
                         fake_features_7, fake_features_8, fake_features_9, fake_features_10, fake_features_11, fake_features_12,
                               fake_features_13, fake_features_14, fake_features_15))
fake_labels = np.concatenate((y_1, y_2, y_3, y_4, y_5, y_6, y_7, y_8, y_9, y_10, y_11, y_12, y_13, y_14, y_15))

fake_features = torch.Tensor(fake_features)

In [10]:
_, preds = torch.max(softmax(model(fake_features.float())), dim = 1)
print(metrics.confusion_matrix((fake_labels), preds.cpu()))
print(metrics.classification_report((fake_labels), preds.cpu(), digits = 5))

[[20  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 20  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  6  0  0  3  0  0  0  0  0  4  0  0  7]
 [ 0  0  0 19  0  0  0  0  0  1  0  0  0  0  0]
 [ 0  2  0  0 16  0  0  0  0  0  2  0  0  0  0]
 [ 0  0  5  0  0  7  0  0  2  0  0  2  0  0  4]
 [ 0  0  0  0  0  0 20  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 12  0  0  8  0  0  0  0]
 [ 0  0  0  0  0  4  0  0 10  0  0  2  0  0  4]
 [ 0  1  0  0  0  0  0  0  0  9  7  0  1  2  0]
 [ 0  0  0  1  4  0  0  2  0  0 13  0  0  0  0]
 [ 0  0  0  0  0  9  0  0  3  0  0  8  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  1  0  0 19  0  0]
 [ 0  0  0  0  0  0  0  1  0  0  4  0  0 15  0]
 [ 0  0  9  0  0  3  0  0  2  0  0  0  0  0  6]]
              precision    recall  f1-score   support

         0.0    1.00000   1.00000   1.00000        20
         1.0    0.86957   1.00000   0.93023        20
         2.0    0.30000   0.30000   0.30000        20
         3.0    0.95000   0.95000   0.95000        20
        